<a href="https://colab.research.google.com/github/AnIsAsPe/Algebra_lineal-para-ML/blob/main/Notebooks/04%20BOW_Ejemplo_de_jugete_espa%C3%B1ol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

#preprocesar texto
import nltk
from nltk.corpus import stopwords
from nltk import SnowballStemmer
import re

nltk.download('stopwords')
nltk.download('punkt')

#construccion de bolsa de palabras

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Creación de Corpus

In [8]:
doc = {}
doc[1] = 'El sol sale para todos y todos los días sale el sol'
doc[2] = 'Estoy en la vereda del sol'
doc[3] = 'Todos ven el sol salir'
doc[4] = 'La vereda sale cara'
doc[5] = 'De cara al sol'

corpus = pd.DataFrame(data=doc.values(), index=doc.keys(), columns=['documento'])
print(corpus.shape)
corpus

(5, 1)


,documento
1,El sol sale para todos y todos los días sale e...
2,Estoy en la vereda del sol
3,Todos ven el sol salir
4,La vereda sale cara
5,De cara al sol


# Preprocesar

In [12]:
def preprocesar(texto):
  #convierte a minúsculas
  texto = texto.lower()

  #quitar tildes 
  import unicodedata  #para quitar tildes
  texto =''.join((c for c in unicodedata.normalize('NFD',texto) if
                         unicodedata.category(c) != 'Mn'))

  #elimina stopwords
  stop = re.compile(r'\b(' + r'|'.join(stopwords.words('spanish')) + r')\b\s*')
  texto = stop.sub('', texto) 

  #stemmer 
  tokens =  nltk.word_tokenize(texto)
  stemmer = SnowballStemmer('spanish')
  stems = [stemmer.stem(token) for token in tokens]
  texto=' '.join(stems) 

  return(texto)

In [13]:
corpus['documento_pp'] = corpus['documento'].apply(preprocesar)
corpus

,documento,documento_pp
1,El sol sale para todos y todos los días sale e...,sol sal dias sal sol
2,Estoy en la vereda del sol,vered sol
3,Todos ven el sol salir,ven sol sal
4,La vereda sale cara,vered sal car
5,De cara al sol,car sol


In [15]:
# Bolsa de palabras pesos binarios
cv = CountVectorizer(binary=True )
mdt_binarios = cv.fit_transform(corpus['documento_pp']) 
palabras = cv.get_feature_names()
bow_binaria = pd.DataFrame(mdt_binarios.todense(), 
                              index=corpus.index, 
                              columns=palabras)
print(bow_binaria.shape)
bow_binaria

(5, 6)


,car,dias,sal,sol,ven,vered
1,0,1,1,1,0,0
2,0,0,0,1,0,1
3,0,0,1,1,1,0
4,1,0,1,0,0,1
5,1,0,0,1,0,0


In [16]:
# Bolsa de palabras pesos frecuencias
cv = CountVectorizer()
mdt_frec = cv.fit_transform(corpus['documento_pp']) 
palabras = cv.get_feature_names()
bow_frec = pd.DataFrame(mdt_frec.todense(), 
                              index=corpus.index, 
                              columns=palabras)
print(bow_frec.shape)
bow_frec
bow_frec

(5, 6)


,car,dias,sal,sol,ven,vered
1,0,1,2,2,0,0
2,0,0,0,1,0,1
3,0,0,1,1,1,0
4,1,0,1,0,0,1
5,1,0,0,1,0,0


In [17]:
# Bolsa de palabras pesos TF-IDF
cv = TfidfVectorizer()
mdt_tfidf = cv.fit_transform(corpus['documento_pp']) 
palabras = cv.get_feature_names()
bow_tfidf = pd.DataFrame(mdt_tfidf.todense(), 
                              index=corpus.index, 
                              columns=palabras)
print(bow_tfidf.shape)
bow_tfidf

(5, 6)


,car,dias,sal,sol,ven,vered
1,0.000000,0.496068,0.664446,0.558953,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.572526,0.000000,0.819887
3,0.000000,0.000000,0.503968,0.423954,0.752515,0.000000
4,0.609818,0.000000,0.506204,0.000000,0.000000,0.609818
5,0.819887,0.000000,0.000000,0.572526,0.000000,0.000000
